# Section 1.1: Subset Datasets

The follwing scripts automates the clipping and manage folder structure based on the HUCs Boundary. You can modify the input list of the datasets,however we recommend to have all the datasets listed below to follow the FAU (2020). We also recomend to have the same variables name for the list of the datasets so that it can be used to rename the datasets to maintain the consistancy and smooth the processing else you might need to debug manually. 

To use the provided script for clipping and mosaicking raster datasets based on HUC boundaries, follow these guidelines:
#Set Project Directory: Update the project_dir variable to point to the root directory of your project where the geodatabase and raster datasets are located.

#Configure HUC Boundary Settings:

gdb_name: Replace "Homestead" with the name of your geodatabase (without the .gdb extension).
huc_fname: Specify the name of the feature class that contains your HUC boundaries.
huc_col: Set this to the column name in your feature class that represents the HUC names.
id_col: Set this to the column name in your feature class that represents the HUC IDs.
Define Dataset Paths:

#datasets: Update this dictionary with the correct paths to your raster datasets. Ensure each key is a descriptive name and each value is the full path to the corresponding raster file.
Running the Script:

Ensure that ArcGIS and its Python environment are properly set up and that you have the necessary permissions to read from and write to the specified directories.
The script will check the number of rows in the HUC feature class. If there is more than one row, it will create subdirectories under HUC12s for each HUC and clip the raster datasets accordingly. If only one row is present, it will clip the datasets and save them in a basedir directory.
After clipping, the script groups the clipped raster files by their names and extensions, creates mosaics for each group, and saves them in the mosaic directory.

In [17]:
import os
import glob
from arcpy.sa import *
arcpy.env.addOutputsToMap = False
# Enable overwriting of output files
arcpy.env.overwriteOutput = True
# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")
# Set the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'

# Settings for the HUC boundary
gdb_name = "Homestead"
huc_fname = "HUC12s_Boundary"  # Name of the feature class
huc_col = "NAME"  # Column representing the name of the HUCs
id_col = "HUC12"  # Column representing the ID of the HUCs

# Replace the paths of the datasets (please check the data management guide for details)
datasets = {
    'dem_ft': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\dem_ft_mosaic.tif',
    'wte_ft': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\wte_ft.tif',
    'whc_ratio': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\whc.tif',
    'impervious': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\impervious.tif',
    'water': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\water.tif',
    'rain_1d5yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_5y1d_in.tif',
    'rain_1d10yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_10y1d_in.tif',
    'rain_1d100yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_25y3d_in.tif',
    'rain_3d25yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_100y1d_in.tif'
}

# Enable overwriting of output files
arcpy.env.overwriteOutput = True

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Set workspace to the project's directory
arcpy.env.workspace = project_dir

# Path to the geodatabase
gdb_path = os.path.join(project_dir, f"{gdb_name}.gdb")
huc_path = os.path.join(gdb_path, huc_fname)

# Function to create directories if they don't exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to clip and save rasters
def clip_and_save(huc, output_dir):
    saved_paths = []
    for key, raster in datasets.items():
        output_raster = os.path.join(output_dir, f"{key}.tif")
        try:
            # Perform the extraction
            arcpy.sa.ExtractByMask(raster, huc).save(output_raster)
            # Check if the raster was saved successfully
            if os.path.exists(output_raster):
                saved_paths.append(output_raster)
                print(f"Clipped {key} successfully and saved to {output_raster}")
            else:
                print(f"Failed to save {key} to {output_raster}")
        except Exception as e:
            print(f"Error clipping {key}: {e}")
    return saved_paths

# Read the feature class and count the number of rows
with arcpy.da.SearchCursor(huc_path, ["SHAPE@", huc_col, id_col]) as cursor:
    rows = [row for row in cursor]

# Check the number of rows in the feature class
if len(rows) > 1:
    huc12s_dir = os.path.join(project_dir, "HUC12s")
    create_dir(huc12s_dir)
    
    all_saved_paths = []  # List to collect all saved paths

    for row in rows:
        # Extract HUC name and ID
        hucname = str(row[1]).title()
        hucid = str(row[2])
        # Define the output subdirectory name based on the HUC ID and HUC name
        subdir = hucid + '_' + hucname.replace(' ', '')
        output_folder = os.path.join(huc12s_dir, subdir)
        create_dir(output_folder)
        # Clip and save the rasters
        saved_paths = clip_and_save(row[0], output_folder)
        all_saved_paths.extend(saved_paths)  # Collect all paths

    # Define the mosaic output folder
    mosaic_dir = os.path.join(project_dir, "mosaic")
    create_dir(mosaic_dir)

    # Group files based on filename and extension
    file_groups = {}
    for file_path in all_saved_paths:
        file_name, file_ext = os.path.splitext(os.path.basename(file_path))
        if (file_name, file_ext) not in file_groups:
            file_groups[(file_name, file_ext)] = []
        file_groups[(file_name, file_ext)].append(file_path)

    # Mosaic each group separately
    for group, files in file_groups.items():
        mosaic_name = f"{group[0]}{group[1]}"
        print(f'Creating mosaic for {mosaic_name}')
        mosaic_path = os.path.join(mosaic_dir, mosaic_name)
        arcpy.MosaicToNewRaster_management(files, mosaic_dir, mosaic_name, pixel_type="32_BIT_FLOAT", number_of_bands=1)
    
else:
    basedir = os.path.join(project_dir, "basedir")
    create_dir(basedir)
    # Clip and save the rasters for the single HUC
    saved_paths = clip_and_save(rows[0][0], basedir)
    print(f"Saved paths: {saved_paths}")

# Clean up
arcpy.CheckInExtension("Spatial")


Clipped dem_ft successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\dem_ft.tif
Clipped wte_ft successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\wte_ft.tif
Clipped whc_ratio successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\whc_ratio.tif
Clipped impervious successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\impervious.tif
Clipped water successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\water.tif
Clipped rain_1d5yr_in successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\rain_1d5yr_in.tif
Clipped rain_1d10yr_in successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\rain_1d10yr_in.tif
Clippe

'CheckedOut'

# Section 1.2 Processing ArcHydro

The second section involves processing ArcHydro tools to obtain the initial stage and time of concentration for the basins, generating intermediate files such as drainage lines, drainage points, and more. This step requires the input datasets, mainly DEM files, to be organized in a specified folder structure.

1.Folder Structure and Input Preparation
#Ensure Proper Folder Structure:
The DEM files should be organized under the HUC12s directory in subdirectories named after their respective HUC IDs. The structure should look like this:


Project_Directory/HUC12s/{HUC_ID}_{HUC_Name}/dem_ft.tif

Set Up Project Directory:
Define your project directory where the input DEM files are located and where the outputs will be stored it should be same project directory which has been used in first section.

2.Create Geodatabase:
Ensure the (ArcHydro.gdb) geodatabase exists within the project directory to store the outputs. If the geodatabase doesn't exist, it will be created by the script.

3.Processing Steps
#Locate DEM Files:
The script uses the glob library to search for DEM files in the specified directory structure.

#Process DEM Files:
For each DEM file found, the script performs the following steps using ArcHydro tools:

Calculate Flow Direction
Calculate Flow Accumulation
Define Streams using a specified threshold
Perform Stream Segmentation
Calculate Stream Order
Create Catchment Grid
Convert Catchment Grid to Catchment Polygon
Generate Drainage Lines
Generate Drainage Points
Save Results:
The output rasters and vector files are saved in the ArcHydro.gdb geodatabase within the project directory. Each output file is named based on the HUC ID extracted from the subdirectory names.

Clean Up:
The script ensures that temporary in-memory data is cleaned up after processing each DEM file.


In [1]:
import os
import glob
import arcpy
from arcpy.sa import *

arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Set the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'

# Path to the geodatabase
gdb_path = os.path.join(project_dir, 'ArcHydro.gdb')

# Create the geodatabase if it doesn't exist
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(project_dir, 'ArcHydro.gdb')
    print(f"Geodatabase created at {gdb_path}")
else:
    print(f"Geodatabase already exists at {gdb_path}")

# Use the glob library to search for dem_ft.tif files in the specified pattern
dem_files = glob.glob(os.path.join(project_dir, 'HUC12s/**/dem_ft.tif'), recursive=True)

# Function to extract numeric part of HUC ID from directory name
def extract_hucid(directory):
    return ''.join(filter(str.isdigit, os.path.basename(directory)))

# Print the list of .tif files
for dem_file in dem_files:
    # Split the file path into directory and file name
    directory, file_name = os.path.split(dem_file)
    hucid = extract_hucid(directory)

    # Generate output names based on hucid
    fd_name = f"{hucid}_fd"
    fa_name = f"{hucid}_fa"
    sd_name = f"{hucid}_sd"
    sl_name = f"{hucid}_sl"
    so_name = f"{hucid}_so"
    cg_name = f"{hucid}_cg"
    cp_name = f"{hucid}_cp"
    dl_name = f"{hucid}_dl"
    dp_name = f"{hucid}_dp"

    try:
        # Step 1: Calculate Flow Direction
        flow_direction = FlowDirection(dem_file, "NORMAL")

        # Step 2: Calculate Flow Accumulation
        flow_accumulation = FlowAccumulation(flow_direction)

        # Step 3: Define Streams (Optional)
        # Adjust the threshold value as needed to define streams
        stream_definition = Con(flow_accumulation > 200, 5)

        # Step 4: Stream Segmentation
        stream_link = StreamLink(flow_direction, stream_definition)
        stream_order = StreamOrder(stream_link, flow_direction)

        # Step 5: Create Catchment Grid
        catchment_grid = Watershed(flow_direction, stream_link)

        # Step 6: Create Catchment Polygon
        catchment_polygon = arcpy.RasterToPolygon_conversion(catchment_grid, "in_memory/catchment_polygons", "NO_SIMPLIFY")

        # Step 7: Create Drainage Line
        drainage_line = StreamToFeature(stream_link, flow_direction, "in_memory/drainage_lines")

        # Step 8: Create Drainage Point
        drainage_point = arcpy.RasterToPoint_conversion(stream_link, "in_memory/drainage_points")

        """# Save the results to the geodatabase
        #flow_direction.save(os.path.join(gdb_path, fd_name))
        flow_accumulation.save(os.path.join(gdb_path, fa_name))
        stream_definition.save(os.path.join(gdb_path, sd_name))
        stream_link.save(os.path.join(gdb_path, sl_name))
        stream_order.save(os.path.join(gdb_path, so_name))
        catchment_grid.save(os.path.join(gdb_path, cg_name))
        """
        arcpy.CopyFeatures_management(catchment_polygon, os.path.join(gdb_path, cp_name))
        arcpy.CopyFeatures_management(drainage_line, os.path.join(gdb_path, dl_name))
        arcpy.CopyFeatures_management(drainage_point, os.path.join(gdb_path, dp_name))

        # Clean up temporary in-memory data
        arcpy.Delete_management("in_memory")
        print(f'Processed {dem_file}')

    except Exception as e:
        print(f"Error processing {dem_file}: {e}")

print("Hydrological analysis completed.")


Geodatabase already exists at X:\amandal2023$\Working\Homestead\Homestead_City\ArcHydro.gdb
Error processing X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\dem_ft.tif: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CopyFeatures).

Error processing X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\dem_ft.tif: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CopyFeatures).

Error processing X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\dem_ft.tif: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CopyFeatures).

Hydrological analysis completed.


# Section 1.3 Unsaturated zone and Soil Storage calculations

Set Project Directory:

Ensure the project_dir variable is correctly set to your project directory path. For example:
python
Copy code
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
Run the Script:

The script processes each DEM file and its corresponding rasters (water table elevation, water holding capacity, impervious, and water) to calculate the unsaturated zone depth and soil storage capacity.
The script automatically saves the output rasters in the appropriate subdirectories.
Review and Adjust Variables if Necessary:

The script derives input file paths based on the dem_ft.tif file paths. Ensure your files are named and organized as per the directory structure above.
If your files have different names, you may need to adjust the replace statements accordingly:
python
Copy code
WTEft = dem_file.replace('dem_ft', 'wte_ft')
WHC = dem_file.replace('dem_ft', 'whc_ratio')
ImperviousBinary = dem_file.replace('dem_ft', 'impervious')
WaterBinary = dem_file.replace('dem_ft', 'water')
Output Files:

The output files for soil storage and unsaturated zone calculations are saved in the same directories as the input dem_ft.tif files, with the names soilStorage and unsaturated, respectively.
Mosaic Output:

The script collects all processed rasters and groups them based on filename and extension.
It then mosaics each group of rasters and saves the results in the mosaic directory.
Ensure the mosaic directory exists or is created by the script.

In [3]:
import os
import glob
import arcpy
from arcpy.sa import *

arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Set the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'


all_saved_paths = []  # List to collect all saved paths
# Print the list of .tif files
for dem_file in dem_files:
    
    
    try:

        # Replace directory names to get corresponding file paths
        WTEft = dem_file.replace('dem_ft', 'wte_ft')
        WHC = dem_file.replace('dem_ft', 'whc_ratio')
        ImperviousBinary = dem_file.replace('DEMft', 'impervious')
        WaterBinary = dem_file.replace('dem_ft', 'water')

        # Define output file paths
        soilstorage = dem_file.replace('dem_ft', 'soilStorage')
        unsaturated = dem_file.replace('dem_ft', 'unsaturated')

        # Calculate soil storage
        soil_outras = RasterCalculator([dem_file, WTEft, WHC, ImperviousBinary, WaterBinary, 12],
                                       ["a", "b", "d", "e", "f", "g"], 
                                       "float((a-b)*d*e*f*g)")
        soil_outras.save(soilstorage)

        # Apply conditional to soil storage
        outdata = Con(soilstorage, soilstorage, 0, "VALUE >0")
        outdata.save(soilstorage)

        # Calculate unsaturated storage
        un_outras = RasterCalculator([dem_file, WTEft], ["a", "b"], "float(a-b)")
        un_outras.save(unsaturated)

        # Apply conditional to unsaturated storage
        outdata = Con(unsaturated, unsaturated, 0, "VALUE >0")
        outdata.save(unsaturated)
        print(f'Processed {dem_file}')
        all_saved_paths.extend([soilstorage,unsaturated])  # Collect all paths

    except Exception as e:
        print(f"Error processing {dem_file}: {e}")
# Define the mosaic output folder
mosaic_dir = os.path.join(project_dir, "mosaic")
#create_dir(mosaic_dir)

# Group files based on filename and extension
file_groups = {}
for file_path in all_saved_paths:
    file_name, file_ext = os.path.splitext(os.path.basename(file_path))
    if (file_name, file_ext) not in file_groups:
        file_groups[(file_name, file_ext)] = []
    file_groups[(file_name, file_ext)].append(file_path)

# Mosaic each group separately
for group, files in file_groups.items():
    mosaic_name = f"{group[0]}{group[1]}"
    print(f'Creating mosaic for {mosaic_name}')
    mosaic_path = os.path.join(mosaic_dir, mosaic_name)
    arcpy.MosaicToNewRaster_management(files, mosaic_dir, mosaic_name, pixel_type="32_BIT_FLOAT", number_of_bands=1)

print("Processing completed.")


Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\dem_ft.tif
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\dem_ft.tif
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\dem_ft.tif
Creating mosaic for soilStorage.tif
Creating mosaic for unsaturated.tif
Processing completed.


# Section 1.4 Input table for Cascade

The final section of the input preparation stage involves summarizing required input parameters into an Excel table for simulating hydrological models based on different design storms. The design rainfall column can vary depending on the design storm events listed in the parent folder or in the data list mentioned in the first section. Similarly, soil storage capacity (SSC) can have multiple columns if we model for various scenarios, as SSC values are influenced by factors like sea level rise and king tides. This section generates an Excel file containing the summarized information with the following headings:

HUC: Hydrologic Unit Code
MaxElevation: Maximum elevation of the DEM
MinElevation: Minimum elevation of the DEM
r1d5yr: Mean rainfall for 1-day, 5-year design storm
r1d10yr: Mean rainfall for 1-day, 10-year design storm
r1d100yr: Mean rainfall for 1-day, 100-year design storm
r3d25yr: Mean rainfall for 3-day, 25-year design storm
SCC0ft: Soil Storage Capacity at 0 feet
SCC1ft: Soil Storage Capacity at 1 foot
SCC2ft: Soil Storage Capacity at 2 feet
SCC3ft: Soil Storage Capacity at 3 feet
SCC4ft: Soil Storage Capacity at 4 feet
SCC5ft: Soil Storage Capacity at 5 feet

In [6]:
import os
import glob
import pandas as pd
from arcpy.sa import *
arcpy.env.addOutputsToMap = False

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

# Enable overwriting of output files
arcpy.env.overwriteOutput = True

# Define the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
gdb_path = os.path.join(project_dir, 'ArcHydro.gdb')

def stat(ras):
    minv = arcpy.GetRasterProperties_management(ras, "MINIMUM")
    maxv = arcpy.GetRasterProperties_management(ras, "MAXIMUM")
    meanv = arcpy.GetRasterProperties_management(ras, "MEAN")
    return [float(minv.getOutput(0)), float(maxv.getOutput(0)), float(meanv.getOutput(0))]

# Use the glob library to search for .tif files in subdirectories named "dem_ft.tif"
dem_files = glob.glob(os.path.join(project_dir, 'HUC12s/**/dem_ft.tif'), recursive=True)

# Define the output directory for the Cascade input file
cascade_output_dir = os.path.join(project_dir, 'Cascade', 'Inputs', 'table')
os.makedirs(cascade_output_dir, exist_ok=True)

# Use the base directory name for the Cascade input file name
base_dir_name = os.path.basename(project_dir)
cascadeinputs = os.path.join(cascade_output_dir, f'{base_dir_name}_Cascade.xlsx')

# Define the temp directory for scc outputs
temp_scc_dir = os.path.join(project_dir, 'Temp', 'scc')
os.makedirs(temp_scc_dir, exist_ok=True)

df_list = []

print(f'Processing directory: {base_dir_name}')
for dem_file in dem_files:
    dirs, filename = os.path.split(dem_file)
    HUC = dirs.split(os.sep)[-1].split('_')[0]  # Extract HUC ID from directory name
    
    dem = stat(dem_file)
    WTEft = dem_file.replace('dem_ft.tif', 'wte_ft.tif')
    WHC = dem_file.replace('dem_ft.tif', 'whc_ratio.tif')
    ImperviousBinary = dem_file.replace('dem_ft.tif', 'impervious.tif')
    WaterBinary = dem_file.replace('dem_ft.tif', 'water.tif')
    
    r1d5yr = stat(dem_file.replace('dem_ft.tif', 'rain_1d5yr_in.tif'))[2]
    r1d10yr = stat(dem_file.replace('dem_ft.tif', 'rain_1d10yr_in.tif'))[2]
    r1d100yr = stat(dem_file.replace('dem_ft.tif', 'rain_1d100yr_in.tif'))[2]
    r3d25yr = stat(dem_file.replace('dem_ft.tif', 'rain_3d25yr_in.tif'))[2]
    
    scclist = []
    for i in range(0, 6):
        output_raster = os.path.join(temp_scc_dir, f'{HUC}_{i}.tif')
        outras = RasterCalculator([dem_file, WTEft, i, WHC, ImperviousBinary, WaterBinary, 12],
                                  ["a", "b", "c", "d", "e", "f", "g"], 
                                  "float((a-(b+c))*d*e*f*g)")
        outras.save(output_raster)
        outdata = Con(output_raster, output_raster, 0, "VALUE >0")
        outdata.save(output_raster)
        scc = stat(output_raster)[2]
        scclist.append(scc)
    
    # Append the row to the list of dictionaries
    df_list.append({'HUC': HUC, 'MaxElevation': dem[1], 'MinElevation': dem[0], 'r1d5yr': r1d5yr,
                    'r1d10yr': r1d10yr, 'r1d100yr': r1d100yr, 'r3d25yr': r3d25yr,
                    'SCC0ft': scclist[0], 'SCC1ft': scclist[1],
                    'SCC2ft': scclist[2], 'SCC3ft': scclist[3], 'SCC4ft': scclist[4], 'SCC5ft': scclist[5]})

# Convert the list of dictionaries to a DataFrame and save to Excel
df = pd.DataFrame(df_list)
df.to_excel(cascadeinputs, index=False)
print(f'Completed processing for directory: {base_dir_name}')

print("Processing completed.")


Processing directory: Homestead_City
Completed processing for directory: Homestead_City
Processing completed.


In [8]:
import os
import glob
from arcpy.sa import *
arcpy.env.addOutputsToMap = False
# Enable overwriting of output files
arcpy.env.overwriteOutput = True
# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")
# Define the directory where you want to start searching for .shp files
start_directory = r'T:\amandal2023$\Working'
ground_water=r'T:\thindle2016$\GW_Surfaces\GW_Broward\data\wte_ft.tif'
outras='Hydro'
outvector='Drainage'
# Use the glob library to search for .tif files in subdirectories named "DEM"
dem_files = glob.glob(os.path.join(start_directory, '**/DEM/*.tif'), recursive=True)
for dem_file in dem_files:
    #print(dem_file)
    a,b=os.path.split(dem_file)
    print(b)
    dp=(dem_file.replace('DEM',outvector)).replace('_ft.tif','_dp.shp')
    dl=(dem_file.replace('DEM',outvector)).replace('_ft.tif','_dl.shp')
    #print(dp, dl)
    # Output feature class with elevation values
    dp_ele = dp.replace('_dp.shp','_dph.shp')
    # Extract elevation values to points
    ExtractValuesToPoints(dp, ground_water, dp_ele)
    # Initialize variables to store minimum elevation and corresponding point
    min_elevation = None
    min_point = None

    # Open a search cursor to iterate through the points
    with arcpy.da.SearchCursor(dp_ele, ["RASTERVALU", "SHAPE@XY"]) as cursor:
        for row in cursor:
            elevation = row[0]
            point = row[1]

            # Check if this elevation is lower than the current minimum
            if min_elevation is None or elevation < min_elevation:
                min_elevation = elevation
                min_point = point

    # Print the coordinates (x, y) of the point with the lowest elevation
    print(f"Point with Lowest Elevation (x, y): {min_point}")
    print(f"Lowest Elevation Value: {min_elevation}")

BISCAYNE_CANAL_ft.tif
Point with Lowest Elevation (x, y): (-80.19337122203126, 25.90972254149375)
Lowest Elevation Value: 0.041062500327825546
LITTLE_RIVER_CANAL_ft.tif
Point with Lowest Elevation (x, y): (-80.18937197316416, 25.85288505738805)
Lowest Elevation Value: 0.0008676649886183441
MIAMI_RIVER_ft.tif
Point with Lowest Elevation (x, y): (-80.38556492230742, 25.81665342921218)
Lowest Elevation Value: -9999.0
SNAKE_CREEK_CANAL_ft.tif
Point with Lowest Elevation (x, y): (-80.29971568390573, 25.956707854258184)
Lowest Elevation Value: 0.4056850075721741
BISCAYNE_CANAL_ft.tif
Point with Lowest Elevation (x, y): (-80.19337122202796, 25.909722541487383)
Lowest Elevation Value: 0.041062500327825546
LITTLE_RIVER_CANAL_ft.tif
Point with Lowest Elevation (x, y): (-80.18937197316416, 25.85288505738805)
Lowest Elevation Value: 0.0008676649886183441
NORTH_BISCAYNE_BAY_ft.tif
Point with Lowest Elevation (x, y): (-80.15938358578103, 25.746257901143508)
Lowest Elevation Value: -9999.0


clip vector

In [24]:
dlist={FlowLine:'Fl',SurfaceStations:'Sp'}
import os
import glob
from arcpy.sa import *
arcpy.env.addOutputsToMap = False
# Enable overwriting of output files
arcpy.env.overwriteOutput = True
# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")
# Define the directory where you want to start searching for .shp files
start_directory = r'T:\amandal2023$\Working'
hucname="NAME"
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
for dirs, data in dlist.items():
    
    # Print the list of .shp files
    for shp_file in shp_files:
        #print(shp_file)
        print(dirs)
        # Split the file path into directory and file name
        directory, file_name = os.path.split(shp_file)
        print(directory)
        output_folder=directory.replace('Huc12\StatePlane','Vector')
        # Make sure the output folder exists, or create it if it doesn't
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        # Create a cursor to iterate through the rows of the shapefile
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname]) as cursor:
            for row in cursor:
                # Define the output raster name based on the OID or any unique identifier
                clipped = str(row[1].replace(' ','_'))+f'_{data}.shp'
                cliped_wte=os.path.join(output_folder, clipped)
                print(cliped_wte)
                # Perform the clip operation
                arcpy.analysis.Clip(dirs, row[0], cliped_wte)
    

T:\amandal2023$\Miami-Dade\Data\Drainage_Lines.shp
T:\amandal2023$\Working\Hialeah\Huc12\StatePlane
T:\amandal2023$\Working\Hialeah\Vector\SNAKE_CREEK_CANAL_Fl.shp
T:\amandal2023$\Working\Hialeah\Vector\LITTLE_RIVER_CANAL_Fl.shp
T:\amandal2023$\Working\Hialeah\Vector\BISCAYNE_CANAL_Fl.shp
T:\amandal2023$\Working\Hialeah\Vector\MIAMI_RIVER_Fl.shp
T:\amandal2023$\Miami-Dade\Data\Drainage_Lines.shp
T:\amandal2023$\Working\NorthMiami\Huc12\StatePlane
T:\amandal2023$\Working\NorthMiami\Vector\NORTH_BISCAYNE_BAY_Fl.shp
T:\amandal2023$\Working\NorthMiami\Vector\LITTLE_RIVER_CANAL_Fl.shp
T:\amandal2023$\Working\NorthMiami\Vector\BISCAYNE_CANAL_Fl.shp
T:\amandal2023$\Miami-Dade\Data\Miami_Stations.shp
T:\amandal2023$\Working\Hialeah\Huc12\StatePlane
T:\amandal2023$\Working\Hialeah\Vector\SNAKE_CREEK_CANAL_Sp.shp
T:\amandal2023$\Working\Hialeah\Vector\LITTLE_RIVER_CANAL_Sp.shp
T:\amandal2023$\Working\Hialeah\Vector\BISCAYNE_CANAL_Sp.shp
T:\amandal2023$\Working\Hialeah\Vector\MIAMI_RIVER_Sp.shp
T

functions